In [1]:
#partition by datetime

In [1]:
import pyspark
import boto3
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, TimestampType
from pyspark.sql.functions import year, to_date, month, dayofmonth,  from_unixtime, unix_timestamp


In [2]:
# ip and environments
environment = 'prd'

# Set the bucket and folder paths
source_bucket = environment + '-landing-zone'
source_folder = 'files/pscore/ascll'

lakehouse_bucket = environment + '-lakehouse'
lakehouse_folder = 'iceberg'

# table destination settings
dest_db_catalog = 'iceberg'
dest_db_schema = 'pscore'
dest_db_table = 'sgw_3'
dest_final_db = f'{dest_db_catalog}.{dest_db_schema}'
dest_final_table = f'{dest_final_db}.{dest_db_table}'

# Spark identification and settings
appname = 'SGW_from_raw_to_raw_refined'
log_level = 'WARN' # Valid log levels include: ALL, DEBUG, ERROR, FATAL, INFO, OFF, TRACE, WARN

# Set your MinIO credentials
s3_endpoint = 'http://minio:9000'
s3_access_key = 'minio'
s3_secret_key = 'minio123'

In [3]:
spark = SparkSession.builder\
    .appName(appname)\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/14 10:33:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark.sparkContext.setLogLevel(log_level)
print(pyspark.SparkConf().getAll())

[('spark.hadoop.hive.cli.print.header', 'true'), ('spark.hadoop.fs.s3a.connection.ssl.enabled', 'false'), ('spark.hadoop.fs.s3a.path.style.access', 'true'), ('spark.sql.catalog.spark_catalog', 'org.apache.iceberg.spark.SparkSessionCatalog'), ('spark.hadoop.fs.s3.endpoint', 'http://minio:9000'), ('spark.app.name', 'SGW_from_raw_to_raw_refined'), ('spark.hadoop.fs.s3.access.key', 'minio'), ('spark.hadoop.fs.s3.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem'), ('spark.sql.catalog.iceberg.uri', 'thrift://hive-metastore:9083'), ('spark.sql.catalog.iceberg.s3.endpoint', 'http://minio:9000'), ('spark.sql.catalog.iceberg.s3.path-style-access', 'true'), ('spark.hadoop.fs.s3.path.style.access', 'true'), ('spark.hive.metastore.uris', 'thrift://hive-metastore:9083'), ('spark.master', 'local[*]'), ('spark.submit.deployMode', 'client'), ('spark.hadoop.fs.s3a.access.key', 'minio'), ('spark.sql.catalog.iceberg.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO'), ('spark.hadoop.fs.s3.secret.key', 'minio123

In [5]:
schema = StructType([
                    StructField("record_type", StringType()),
                    StructField("network_initiated_pdp_context", StringType()),
                    StructField("imsi", StringType()),
                    StructField("msisdn", StringType()),
                    StructField("imei", StringType()),
                    StructField("charging_id", StringType()),
                    StructField("ggsn_pgw_address", StringType()),
                    StructField("sgsn_sgw_address", StringType()),
                    StructField("ms_nw_capability", StringType()),
                    StructField("pdp_pdn_type", StringType()),
                    StructField("served_pdp_address", StringType()),
                    StructField("dynamic_address_flag", StringType()),
                    StructField("access_point_name_ni", StringType()),
                    StructField("record_sequence_number", StringType()),
                    StructField("record_sequence_number_meg", StringType()),
                    StructField("node_id", StringType()),
                    StructField("local_sequence_number", StringType()),
                    StructField("charging_characteristics", StringType()),
                    StructField("record_opening_time", StringType()),
                    StructField("duration", StringType()),
                    StructField("rat_type", StringType()),
                    StructField("cause_for_record_closing", StringType()),
                    StructField("diagnostic", StringType()),
                    StructField("volume_uplink", StringType()),
                    StructField("volume_downlink", StringType()),
                    StructField("total_volume", StringType()),
                    StructField("lac_or_tac", StringType()),
                    StructField("ci_or_eci", StringType()),
                    StructField("rac", StringType()),
                    StructField("rnc_unsent_data_volume", StringType()),
                    StructField("req_alloc_ret_priority", StringType()),
                    StructField("neg_alloc_ret_priority", StringType()),
                    StructField("req_traffic_class", StringType()),
                    StructField("neg_traffic_class", StringType()),
                    StructField("qci", StringType()),
                    StructField("req_max_bitrate_uplink", StringType()),
                    StructField("req_max_bitrate_downlink", StringType()),
                    StructField("req_guar_bitrate_uplink", StringType()),
                    StructField("req_guar_bitrate_downlink", StringType()),
                    StructField("neg_max_bitrate_uplink", StringType()),
                    StructField("neg_max_bitrate_downlink", StringType()),
                    StructField("neg_guar_bitrate_uplink", StringType()),
                    StructField("neg_guar_bitrate_downlink", StringType()),
                    StructField("mccmnc", StringType()),
                    StructField("country_name", StringType()),
                    StructField("input_filename", StringType()),
                    StructField("output_filename", StringType()),
                    StructField("event_date", TimestampType())
                ])

In [6]:
spark.sql(f"""
CREATE DATABASE IF NOT EXISTS iceberg.raw COMMENT '' LOCATION 's3a://prd-lakehouse/iceberg/raw/'
""")

DataFrame[]

In [8]:
spark.sql(f"""
DROP TABLE IF EXISTS iceberg.raw.{dest_db_table}
""")

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


DataFrame[]

In [9]:
table = spark.sql(f"""
        CREATE TABLE IF NOT EXISTS iceberg.raw.{dest_db_table}
        (
            record_type STRING,
            network_initiated_pdp_context STRING,
            imsi STRING,
            msisdn STRING,
            imei STRING,
            charging_id STRING,
            ggsn_pgw_address STRING,
            sgsn_sgw_address STRING,
            ms_nw_capability STRING,
            pdp_pdn_type STRING,
            served_pdp_address STRING,
            dynamic_address_flag STRING,
            access_point_name_ni STRING,
            record_sequence_number STRING,
            record_sequence_number_meg STRING,
            node_id STRING,
            local_sequence_number STRING,
            charging_characteristics STRING,
            record_opening_time STRING,
            duration STRING,
            rat_type STRING,
            cause_for_record_closing STRING,
            diagnostic STRING,
            volume_uplink STRING,
            volume_downlink STRING,
            total_volume STRING,
            lac_or_tac STRING,
            ci_or_eci STRING,
            rac STRING,
            rnc_unsent_data_volume STRING,
            req_alloc_ret_priority STRING,
            neg_alloc_ret_priority STRING,
            req_traffic_class STRING,
            neg_traffic_class STRING,
            qci STRING,
            req_max_bitrate_uplink STRING,
            req_max_bitrate_downlink STRING,
            req_guar_bitrate_uplink STRING,
            req_guar_bitrate_downlink STRING,
            neg_max_bitrate_uplink STRING,
            neg_max_bitrate_downlink STRING,
            neg_guar_bitrate_uplink STRING,
            neg_guar_bitrate_downlink STRING,
            mccmnc STRING,
            country_name STRING,
            input_filename STRING,
            output_filename STRING,
            event_date Timestamp
        ) 
        USING iceberg
        PARTITIONED BY (event_date)
        """)
        

24/03/14 10:36:04 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [10]:
s3 = boto3.client('s3', endpoint_url=s3_endpoint, aws_access_key_id=s3_access_key, aws_secret_access_key=s3_secret_key)

In [12]:
# List all files in the source directory
file_list = []
paginator = s3.get_paginator('list_objects_v2')

for result in paginator.paginate(Bucket=f"{source_bucket}", Prefix=source_folder):
    
    if 'Contents' in result:
        for item in result['Contents']:
            file_list.append(item['Key'])


In [19]:
# reading files in the source
for file_name in file_list:

    print(f'File in processing: {file_name}')
    
    df = spark.read.format("csv") \
                    .option("header", "false") \
                    .option("delimiter", ";") \
                    .schema(schema) \
                    .load(f"s3a://{source_bucket}/{file_name}")
    # some transformations
    df = df.withColumn("duration", df["duration"].cast("double"))
    # df = df.withColumn("event_date", to_date(df["record_opening_time"], "yyyyMMddHHmmss"))
    # to_date(df["record_opening_time"], "yyyyMMddHHmmss")

    df.withColumn("event_date", from_unixtime(unix_timestamp("record_opening_time", "yyyyMMddHHmmss")))

    df.select('input_filename').show()
    # wrintint the data on lakehouse
    df.writeTo(f'iceberg.raw.{dest_db_table}').append()     


File in processing: files/pscore/ascll/DATA_TRAFFIC_85994_99022_20240312201006.cdr
+--------------------+
|      input_filename|
+--------------------+
|qor-mme21_-_41157...|
|qor-mme21_-_41157...|
|qor-mme21_-_41157...|
|qor-mme21_-_41157...|
|qor-mme21_-_41157...|
|qor-mme21_-_41157...|
|qor-mme21_-_41157...|
|qor-mme21_-_41157...|
|qor-mme21_-_41157...|
|qor-mme21_-_41157...|
|qor-mme21_-_41157...|
|qor-mme21_-_41157...|
|qor-mme21_-_41157...|
|qor-mme21_-_41157...|
|qor-mme21_-_41157...|
|qor-mme21_-_41157...|
|qor-mme21_-_41157...|
|qor-mme21_-_41157...|
|qor-mme21_-_41157...|
|qor-mme21_-_41157...|
+--------------------+
only showing top 20 rows



File in processing: files/pscore/ascll/DATA_TRAFFIC_86084_86352_20240313111006.cdr
+--------------------+
|      input_filename|
+--------------------+
|bir-mme12_-_3758....|
|bir-mme12_-_3758....|
|bir-mme12_-_3758....|
|bir-mme12_-_3758....|
|bir-mme12_-_3758....|
|bir-mme12_-_3758....|
|bir-mme12_-_3758....|
|bir-mme12_-_3758....|
|bir-mme12_-_3758....|
|bir-mme12_-_3758....|
|bir-mme12_-_3758....|
|bir-mme12_-_3758....|
|bir-mme12_-_3758....|
|qor-mme21_-_41216...|
|qor-mme21_-_41216...|
|qor-mme21_-_41216...|
|qor-mme21_-_41216...|
|qor-mme21_-_41216...|
|qor-mme21_-_41216...|
|qor-mme21_-_41216...|
+--------------------+
only showing top 20 rows



File in processing: files/pscore/ascll/DATA_TRAFFIC_86085_32451_20240313113000.cdr
+--------------------+
|      input_filename|
+--------------------+
|qor-mme21_-_41218...|
|qor-mme21_-_41218...|
|qor-mme21_-_41218...|
|qor-mme21_-_41218...|
|qor-mme21_-_41218...|
|qor-mme21_-_41218...|
|qor-mme21_-_41218...|
|qor-mme21_-_41218...|
|qor-mme21_-_41218...|
|qor-mme21_-_41218...|
|qor-mme21_-_41218...|
|qor-mme21_-_41218...|
|qor-mme21_-_41218...|
|qor-mme21_-_41218...|
|qor-mme21_-_41218...|
|qor-mme21_-_41218...|
|qor-mme21_-_41218...|
|qor-mme21_-_41218...|
|qor-mme21_-_41218...|
|qor-mme21_-_41218...|
+--------------------+
only showing top 20 rows



File in processing: files/pscore/ascll/DATA_TRAFFIC_86086_90254_20240313113005.cdr
+--------------------+
|      input_filename|
+--------------------+
|qor-mme21_-_41219...|
|qor-mme21_-_41219...|
|qor-mme21_-_41219...|
|qor-mme21_-_41219...|
|qor-mme21_-_41219...|
|qor-mme21_-_41219...|
|qor-mme21_-_41219...|
|qor-mme21_-_41219...|
|qor-mme21_-_41219...|
|qor-mme21_-_41219...|
|qor-mme21_-_41219...|
|qor-mme21_-_41219...|
|qor-mme21_-_41219...|
|qor-mme21_-_41219...|
|qor-mme21_-_41219...|
|qor-mme21_-_41219...|
|qor-mme21_-_41219...|
|qor-mme21_-_41219...|
|qor-mme21_-_41219...|
|qor-mme21_-_41219...|
+--------------------+
only showing top 20 rows



File in processing: files/pscore/ascll/DATA_TRAFFIC_86087_56086_20240313115000.cdr
+--------------------+
|      input_filename|
+--------------------+
|bir-mme12_-_3769....|
|bir-mme12_-_3769....|
|bir-mme12_-_3769....|
|bir-mme12_-_3769....|
|bir-mme12_-_3769....|
|qor-mme22_-_31406...|
|qor-mme22_-_31406...|
|qor-mme22_-_31406...|
|qor-mme22_-_31406...|
|qor-mme22_-_31406...|
|qor-mme22_-_31406...|
|qor-mme22_-_31406...|
|qor-mme22_-_31406...|
|qor-mme22_-_31406...|
|qor-mme22_-_31406...|
|qor-mme22_-_31406...|
|qor-mme22_-_31406...|
|qor-mme22_-_31406...|
|qor-mme22_-_31406...|
|qor-mme22_-_31406...|
+--------------------+
only showing top 20 rows



In [18]:
tb = spark.table(f'iceberg.raw.{dest_db_table}')
print(tb.printSchema())
print(f"No of Records: {tb.count()}")

root
 |-- record_type: string (nullable = true)
 |-- network_initiated_pdp_context: string (nullable = true)
 |-- imsi: string (nullable = true)
 |-- msisdn: string (nullable = true)
 |-- imei: string (nullable = true)
 |-- charging_id: string (nullable = true)
 |-- ggsn_pgw_address: string (nullable = true)
 |-- sgsn_sgw_address: string (nullable = true)
 |-- ms_nw_capability: string (nullable = true)
 |-- pdp_pdn_type: string (nullable = true)
 |-- served_pdp_address: string (nullable = true)
 |-- dynamic_address_flag: string (nullable = true)
 |-- access_point_name_ni: string (nullable = true)
 |-- record_sequence_number: string (nullable = true)
 |-- record_sequence_number_meg: string (nullable = true)
 |-- node_id: string (nullable = true)
 |-- local_sequence_number: string (nullable = true)
 |-- charging_characteristics: string (nullable = true)
 |-- record_opening_time: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- rat_type: string (nullable = true)
 |-- 